<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/capm/alphas_mve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# example market mean and std dev
mmn = 0.12
msd = 0.20

# example stock std dev and alpha and correlation between stock and market
ssd = 0.30
alpha = 0.05
corr = 0.3

# example risk-free rate
rf = 0.02

# calculate stock beta
beta = corr * ssd / msd

# calculate market and stock risk premia
mprem = mmn - rf
sprem = alpha+beta*mprem
rprem = np.array([mprem,sprem])

# calculate covariance matrix
mvar = msd**2
svar = ssd**2
cov = corr*msd*ssd
cov = np.array([[mvar,cov],[cov,svar]])

# calculate tangency portfolio
tang = np.linalg.solve(cov,rprem)
tang = tang / np.sum(tang)
tang = pd.Series(tang,index=['market','stock'])
tang



market    0.784715
stock     0.215285
dtype: float64